In [114]:
import pandas as pd
import numpy as np


def binomial_transition(n, prob):
    return np.random.binomial(n, prob)

def poisson_transition(n, rate):
    return np.random.poisson(np.nan_to_num(n * rate))

In [213]:

def check_state_space(x, pop=None):
    return np.clip(x, 0, pop)

def f(t, x, gamma, beta, delta, Nmean, N, A, D, M):
    C = x[0, :, :]
    S = np.clip(N - C,0,N)

    λ = beta * C / Nmean # force of infection

    # moving out and in colonized
    Cout  = binomial_transition(list(np.sum(M, axis=1, keepdims=True)), np.clip(np.nan_to_num(C/N), 0, 1))
    Cin   = M.T @ np.clip(np.nan_to_num(C/N), 0, 1)
 
    a2c   = binomial_transition(list(A), gamma)                                  # people admitted colonized.
    c2out = binomial_transition(list(D), np.clip(np.nan_to_num(C/N), 0, 1))  # discharged colonized

    s2c  = poisson_transition(S, λ) # new colonized
    c2s  = poisson_transition(C, delta) # decolonizations

    C    = C + a2c - c2out + s2c + c2s + Cin - Cout
    C    = np.clip(C, 0, N)

    return check_state_space(np.array([C, a2c, s2c]))

def g(t, x, N, rho, tests, model_settings):
    """ Observational model
        Args:
            t (int):      Time
            x (np.array): State space
            rho (float):  Observation probability
        Returns:
            y (np.array): Observed carriers ~ Binomial(C, rho)
    """

    m       = model_settings["m"]
    num_pop = model_settings["num_pop"]

    C       = x[0, :, :]
    N       = np.reshape([N]*m,(m,num_pop)).T
    tests   = np.reshape(list(tests)*m,(m,len(tests))).T

    with np.errstate(divide='ignore', invalid='ignore'):
        observed_colonized = np.random.binomial(tests.astype('int'), rho * np.nan_to_num(C/N))

    return observed_colonized


def f0(N0, c0, model_settings):

    m       = model_settings["m"]
    num_pop = model_settings["num_pop"]

    N0   = np.array([N0]) * np.ones((num_pop, m))
    C0   = c0 * N0

    AC   = np.zeros((num_pop, m))
    newC = np.zeros((num_pop, m))

    return np.array([C0, AC, newC])

In [ ]:
# parameters
γ = 0.05 * np.ones((model_settings["num_pop"], model_settings["m"]))
β = 0.1  * np.ones((model_settings["num_pop"], model_settings["m"]))
δ = 1/120

# the movement model

M is a matrix of [np, np] and C and N vectors of size [np, m].

Mij = individuals moving for i to j.

1. Colonized ppl moving outside unit $i$. The people moving outside of unit i are $\sum_j M_{ij}$.

    $C_{i}^{out}= \sum_j M_{ij} \frac{C_i}{N_i}$

    In code it will be
    ```
    Cout_i = np.sum(M[i, :]) * C[i, :]/N[i, :]
    ```
    So for putting all the people in a matix it will be
    ```
    Cout = np.sum(M, axis=1, keepdims=True) * C / N
    ```
2. Colonized people moving into unit $i$.
   For a 3 metapop model, the people moving into population $i=1$ will be $M_{21}, M_{31}$
   So the people moving inside colonized are $C_1^{out}=M_{21} * C_2/N_2$ and $C_2^{out}=M_{31} * C_3/N_3$.
    As a matrix multiplication this is $M_{:,1}^T \times C/N$, where $M_{:,1}^T=[M_{11}, M_{21}, M_{31}]$ and $C=[C_1, C_2, C_3]^T$.

   The total people moving inside colonized are $C_i^{out}=\sum_j M_{ji} * C_j/N_j$.

   So, in matrix multiplication (I think - not sure) the people moving into all population can be writted as below, gotta be carefull to make the diagonal in M zero!!! or will add people in the same ward.

   $$C_{in} = M^T \times C$$